<a href="https://colab.research.google.com/github/trduc97/Spotify_classification/blob/main/spotify_enrich.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import libraries and datafiles

In [ ]:
import gdown
import pandas as pd

# Define the URL of the CSV file on Google Drive
train_url = 'https://drive.google.com/uc?id=1xta0FaTHYVKjLbc7ztPgDME5nsqZGoPz'
test_url = 'https://drive.google.com/uc?id=1O0DtiXCsgl7CDV52yOvVWwaKriXjqWxL'

# Define the output file name
train_file = 'classification_train.csv'
test_file = 'classification_test.csv'
# Download the file from Google Drive
gdown.download(train_url, train_file, quiet=False)
gdown.download(test_url, test_file, quiet=False)

# Read the CSV file into a DataFrame
train_set = pd.read_csv(train_file)
test_set = pd.read_csv(test_file)



Downloading...
From: https://drive.google.com/uc?id=1xta0FaTHYVKjLbc7ztPgDME5nsqZGoPz
To: /content/classification_train.csv
100%|██████████| 39.1k/39.1k [00:00<00:00, 36.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O0DtiXCsgl7CDV52yOvVWwaKriXjqWxL
To: /content/classification_test.csv
100%|██████████| 8.57k/8.57k [00:00<00:00, 17.2MB/s]


## 1.1. Importing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler
# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# default is 120 seconds
%autosave 60

# showing all columns
pd.set_option('display.max_columns', None)

Autosaving every 60 seconds


In [ ]:
# update the genres for the missing values
train_set.loc[train_set['title'] == 'Unchained Melody', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'Someone Elses Roses', 'top genre'] = 'adult standards'
train_set.loc[train_set['title'] == 'Drinks On The House', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'Pachuko Hop', 'top genre'] = 'blues'
train_set.loc[train_set['title'] == 'Little Things Means A Lot', 'top genre'] = 'blues'
train_set.loc[train_set['title'] == 'The Lady Is A Tramp', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'If I Give My Heart To You', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'Happy Days And Lonely Nights','top genre'] = 'rock'
train_set.loc[train_set['title'] == 'Stairway Of Love','top genre'] = 'rock'
train_set.loc[train_set['title'] == 'You', 'top genre'] = 'pop'
train_set.loc[train_set['title'] == 'No Other Love' , 'top genre'] = 'adult standards'
train_set.loc[train_set['title'] == "Hot Diggity" , 'top genre'] = 'folk'
train_set.loc[train_set['title'] == "Ain't That Just the Way" , 'top genre'] = 'r&b'
train_set.loc[train_set['title'] == "I Promised Myself" , 'top genre'] = 'pop'
# dropping NULL values  (I've Waited So Long Anthony Newley = ? (Dance/ Electronic))
train_set = train_set.dropna(axis=0)

# resetting index to fill out the dropped row
train_set.reset_index(drop=True, inplace=True)
# check again for null values
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         452 non-null    int64 
 1   title      452 non-null    object
 2   artist     452 non-null    object
 3   year       452 non-null    int64 
 4   bpm        452 non-null    int64 
 5   nrgy       452 non-null    int64 
 6   dnce       452 non-null    int64 
 7   dB         452 non-null    int64 
 8   live       452 non-null    int64 
 9   val        452 non-null    int64 
 10  dur        452 non-null    int64 
 11  acous      452 non-null    int64 
 12  spch       452 non-null    int64 
 13  pop        452 non-null    int64 
 14  top genre  452 non-null    object
dtypes: int64(12), object(3)
memory usage: 53.1+ KB


In [ ]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
train_set.rename(columns={'top genre': 'top_genre'}, inplace=True)
train_set.head()

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top_genre
0,1,My Happiness,Connie Francis,1996,107,31,45,-8,13,28,150,75,3,44,adult standards
1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,47,139,49,3,37,pop
2,3,How Deep Is Your Love,Bee Gees,1979,105,36,63,-9,13,67,245,11,3,77,adult standards
3,4,Woman in Love,Barbra Streisand,1980,170,28,47,-16,13,33,232,25,3,67,adult standards
4,5,Goodbye Yellow Brick Road - Remastered 2014,Elton John,1973,121,47,56,-8,15,40,193,45,3,63,glam rock


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

'''
# Duc
cid = '75b8b1605cb64f42bf5705c1e9388076'
client_secret = 'd77064f8876340fa96592c7d001599fa'

# Felix
cid= 'cf6ec50bda354dc282ab13a7533b1d88'
client_secret = '851d005c3baf48118ee9637b55feb301'

# Hang
cid = '25394a9c7ba0433abf4a25ff73f5dea9'
client_secret = '169b175347b7451e81499416006375c6'
'''
# Felix
cid= 'cf6ec50bda354dc282ab13a7533b1d88'
client_secret = '851d005c3baf48118ee9637b55feb301'


# Initialize Spotipy with user credentials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=client_secret))

In [ ]:
artist_name = 'TED'
sp.search(q=artist_name, type='artist', limit=1)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=TED&type=artist&offset=0&limit=1',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6FBQaQCHibmsKWSMKiWfBb'},
    'followers': {'href': None, 'total': 20},
    'genres': [],
    'href': 'https://api.spotify.com/v1/artists/6FBQaQCHibmsKWSMKiWfBb',
    'id': '6FBQaQCHibmsKWSMKiWfBb',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b273b27ebe1f6f9958f0957eaa0d',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e02b27ebe1f6f9958f0957eaa0d',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/ab67616d00004851b27ebe1f6f9958f0957eaa0d',
      'width': 64}],
    'name': 'TEDDY',
    'popularity': 0,
    'type': 'artist',
    'uri': 'spotify:artist:6FBQaQCHibmsKWSMKiWfBb'}],
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?query=TED&type=artist&offset=1&limit=1',
  'offset': 0,
  'previous': None

# 1.crawling from the original playlists of all-out-music
The playlist change constantly so much of the original songs have been changed

In [ ]:
def crawl_playlist(playlist_id, batch_size = 10):
    # Create a list to save all the song data
    song_data = []

    # Create lists to save track and artist ID
    track_ids = []
    artist_ids = []

    # Get playlist tracks
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']

    # Continue fetching tracks until there are no more pages
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    # Collecting the IDs of the playlist
    for track in tracks:
        track_ids.append(track['track']['id'])
        artist_ids.append(track['track']['artists'][0]['id'])  # Assuming only one artist per track

    # Crawling the data of the tracks in batch
    for i in range(0, len(track_ids), batch_size):

        track_id_batch = track_ids[i:i+batch_size]
        artist_id_batch = artist_ids[i:i+batch_size]
        # Get features for the batch of track and artist IDs
        track_info_batch = [sp.track(track_id) for track_id in track_id_batch]
        track_features_batch = sp.audio_features(track_id_batch) # tracks' audio features require calling a whole batch to avoid getting flagged and warning
        artist_features_batch = [sp.artist(artist_id) for artist_id in artist_id_batch]

        # Combine track and artist features
        for track_info, track_features, artist_features in zip(track_info_batch,track_features_batch, artist_features_batch):
            song_data.append({"track_info": track_info,"track_features": track_features, "artist_features": artist_features})

    # Convert song data into a DataFrame for easier manipulation and analysis
    return pd.json_normalize(song_data)

In [ ]:
all_data = pd.DataFrame()

In [ ]:
all_data = pd.read_csv('all_data.csv')

In [ ]:
playlists_id = [
#               '37i9dQZF1DWSV3Tk4GO2fq',
#                '37i9dQZF1DXaKIA8E7WcJj',
#                '37i9dQZF1DWTJ7xPn4vNaz',
#                '37i9dQZF1DX4UtSsGT1Sbe',
#                '37i9dQZF1DXbTxeAdrVG2l',
#                '37i9dQZF1DX4o1oenSJRJd',
                '37i9dQZF1DX5Ejj0EkURtP'
                ]


for id in playlists_id:
  playlist_data = crawl_playlist(id, batch_size = 10)
  all_data = pd.concat([all_data, playlist_data], ignore_index=True)

In [ ]:
all_data.shape

(1050, 59)

In [ ]:
all_data.to_csv('all_data.csv')

In [ ]:
test_col = ['Id', 'title', 'artist', 'year', 'bpm', 'nrgy', 'dnce', 'dB', 'live',
        'val', 'dur', 'acous', 'spch', 'pop']

train_col = ['Id', 'title', 'artist', 'year', 'bpm', 'nrgy', 'dnce', 'dB', 'live',
        'val', 'dur', 'acous', 'spch', 'pop', 'top_genre']

In [ ]:
retained_cols = [
#'track_info.album.album_type',
# 'track_info.album.artists',
 'track_info.album.available_markets',
# 'track_info.album.external_urls.spotify',
# 'track_info.album.href',
# 'track_info.album.id',
# 'track_info.album.images',
 'track_info.album.name',
 'track_info.album.release_date',
# 'track_info.album.release_date_precision',
 'track_info.album.total_tracks',
# 'track_info.album.type',
# 'track_info.album.uri',
 #'track_info.artists',
 'track_info.available_markets',
 'track_info.disc_number',
 'track_info.duration_ms',
 'track_info.explicit',
# 'track_info.external_ids.isrc',
# 'track_info.external_urls.spotify',
# 'track_info.href',
# 'track_info.id',
# 'track_info.is_local',
 'track_info.name',
 'track_info.popularity',
# 'track_info.preview_url',
 'track_info.track_number',
# 'track_info.type',
# 'track_info.uri',
 'track_features.danceability',
 'track_features.energy',
 'track_features.key',
 'track_features.loudness',
 'track_features.mode',
 'track_features.speechiness',
 'track_features.acousticness',
 'track_features.instrumentalness',
 'track_features.liveness',
 'track_features.valence',
 'track_features.tempo',
# 'track_features.track_href',
# 'track_features.analysis_url',
 'track_features.time_signature',
# 'artist_features.external_urls.spotify',
# 'artist_features.followers.href',
 'artist_features.followers.total',
 'artist_features.genres',
# 'artist_features.href',
# 'artist_features.id',
# 'artist_features.images',
 'artist_features.name',
 'artist_features.popularity',
# 'artist_features.type',
# 'artist_features.uri'
  ]

In [ ]:
total_data = all_data[retained_cols].copy()

In [ ]:
def spotify_preporcess(dataset, original_df, retained_cols):
  dataset = dataset[retained_cols]
  # Generating metadata
  dataset['num_artist_genres'] = dataset['artist_features.genres'].apply(lambda x: len(x))
  dataset['num_track_available_markets'] = dataset['track_info.available_markets'].apply(lambda x: len(x))
  dataset['num_album_available_markets'] = dataset['track_info.album.available_markets'].apply(lambda x: len(x))
  print('Generating metadata: ',dataset.shape)
  # Creating release_year col
  dataset['release_year'] = dataset['track_info.album.release_date'].str.extract(r'^(\d{4})')

  # Convert the 'release_year' column to numeric
  dataset.loc[:, 'release_year'] = pd.to_numeric(dataset['release_year'], errors='coerce')
  print('Adding release_year: ', dataset.shape)
  # Merge df_test and train_set based on all three conditions
  matched_rows = dataset.merge(original_df,
                             how='inner',
                             left_on=['artist_features.name',
                                      'release_year',
                                      'track_info.name'
                                      ],
                             right_on=['artist',
                                       'year',
                                       'title'
                                       ])
  print('Matching data based on original dataset: ', matched_rows.shape)
  # Dropping duplicate rows
  matched_rows= matched_rows.drop_duplicates(subset=[
    col for col in matched_rows.columns
    if col not in ['artist_features.genres', 'track_info.available_markets', 'track_info.album.available_markets']])
  print('Dropping duplicate values: ', matched_rows.shape)
  return matched_rows,dataset

In [ ]:
train_matched_r1,total_df = spotify_preporcess(total_data, train_set, retained_cols)

Generating metadata:  (1050, 30)
Adding release_year:  (1050, 31)
Matching data based on original dataset:  (98, 46)
Dropping duplicate values:  (97, 46)


<ipython-input-15-00d7ac0c5e44>:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dataset.loc[:, 'release_year'] = pd.to_numeric(dataset['release_year'], errors='coerce')


In [ ]:
test_matched_r1,total_df = spotify_preporcess(total_data, test_set, retained_cols)

Generating metadata:  (1050, 30)
Adding release_year:  (1050, 31)
Matching data based on original dataset:  (25, 45)
Dropping duplicate values:  (24, 45)


<ipython-input-15-00d7ac0c5e44>:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dataset.loc[:, 'release_year'] = pd.to_numeric(dataset['release_year'], errors='coerce')


# 2.Run search with combine of songs+artist that match the description

In [ ]:
train_r2 = train_set[~train_set['title'].isin(train_matched_r1['title'])].copy()
train_r2.reset_index(drop=True, inplace=True)
train_r2

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top_genre
0,1,My Happiness,Connie Francis,1996,107,31,45,-8,13,28,150,75,3,44,adult standards
1,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,47,139,49,3,37,pop
2,4,Woman in Love,Barbra Streisand,1980,170,28,47,-16,13,33,232,25,3,67,adult standards
3,6,Grenade,Bruno Mars,2010,110,56,71,-7,12,23,223,15,6,74,pop
4,7,No Time,The Guess Who,1971,128,48,48,-14,5,12,219,8,6,44,album rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,448,A World Without Love,Peter And Gordon,2017,130,66,51,-10,11,75,161,23,6,37,brill building pop
351,449,But Not For Me,Ella Fitzgerald,1959,80,22,18,-17,10,16,214,92,4,45,adult standards
352,450,Surf City,Jan & Dean,2010,148,81,53,-13,23,96,147,50,3,50,brill building pop
353,452,It's Gonna Be Me,*NSYNC,2000,165,87,64,-5,6,88,191,5,8,62,boy band


In [ ]:
test_r2 = test_set[~test_set['title'].isin(test_matched_r1['title'])].copy()
test_r2.reset_index(drop=True, inplace=True)
test_r2

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,454,Pump It,The Black Eyed Peas,2005,154,93,65,-3,75,74,213,1,18,72
1,455,"Circle of Life - From ""The Lion King""/Soundtra...",Elton John,1994,161,39,30,-15,11,14,292,26,3,59
2,456,We Are The Champions - Remastered 2011,Queen,1977,64,46,27,-7,12,18,179,38,3,76
3,457,Insomnia - Radio Edit,Faithless,2010,127,92,71,-9,37,53,216,6,4,50
4,458,This Eve of Parting,John Hartford,2018,115,46,56,-12,21,34,153,18,3,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,561,"Rhythm Is A Dancer - 7"" Edit",SNAP!,1992,124,75,79,-11,14,71,226,27,4,67
85,562,"Crockett's Theme - From ""Miami Vice II"" Soundt...",Jan Hammer,1991,191,39,41,-17,10,40,215,42,3,58
86,564,Dragostea Din Tei - Italian Version,O-Zone,2010,130,89,67,-6,10,80,215,4,3,44
87,565,Big Poppa - 2005 Remaster,The Notorious B.I.G.,1994,84,58,78,-7,14,76,253,43,27,74


In [ ]:
def search_song(name, artist, limit=3):
    # Search for the song by name and artist, limit=3 to get top 3 results
    result = sp.search(q=f'{name} {artist} ', type='track', limit=limit)
    items = result['tracks']['items']

    if items:
        return items
    else:
        return []

# List to hold song data

def search_songs_details(test_set, batch_size = 10, limit=3):
    song_data = []

    for i in range(0, len(test_set), batch_size):
        # Get a batch of track IDs and artist IDs
        track_ids = []
        artist_ids = []

        # Prepare batch of track and artist IDs
        for j in range(i, min(i + batch_size, len(test_set))):
            tracks = search_song(test_set['title'][j], test_set['artist'][j], limit= limit)
            for track in tracks:
                track_ids.append(track['id'])
                artist_ids.append(track['artists'][0]['id'])  # Assuming the first artist

        # Get features for the batch of track and artist IDs
        track_info_batch = [sp.track(track_id) for track_id in track_ids]
        track_features_batch = sp.audio_features(track_ids) # tracks' audio features require calling a whole batch to avoid getting flagged and warning
        artist_features_batch = [sp.artist(artist_id) for artist_id in artist_ids]

        # Combine track and artist features
        for track_info, track_features, artist_features in zip(track_info_batch,track_features_batch, artist_features_batch):
            song_data.append({"track_info": track_info,"track_features": track_features, "artist_features": artist_features})

    # Convert song data into a DataFrame for easier manipulation and analysis
    return pd.json_normalize(song_data)

In [ ]:
train_set_r2_modified = search_songs_details(train_r2)

In [ ]:
train_set_r2_modified.to_csv('train_set_r2.csv')

In [ ]:
train_set_r2_modified = pd.read_csv('train_set_r2.csv')

In [ ]:
train_matched_r2_modified, train_r2_raw = spotify_preporcess(train_set_r2_modified, train_r2, retained_cols)

Generating metadata:  (848, 30)
Adding release_year:  (848, 31)
Matching data based on original dataset:  (226, 46)
Dropping duplicate values:  (223, 46)


<ipython-input-15-00d7ac0c5e44>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_artist_genres'] = dataset['artist_features.genres'].apply(lambda x: len(x))
<ipython-input-15-00d7ac0c5e44>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_track_available_markets'] = dataset['track_info.available_markets'].apply(lambda x: len(x))
<ipython-input-15-00d7ac0c5e44>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [ ]:
train_matched_r2_modified['title'].nunique()

189

In [ ]:
test_set_r2_modified = search_songs_details(test_r2)

In [ ]:
test_set_r2_modified.to_csv('test_set_r2.csv')

In [ ]:
test_set_r2_modified = pd.read_csv('test_set_r2.csv')

In [ ]:
test_r2_matched_modified, train_r2_raw= spotify_preporcess(test_set_r2_modified, test_r2, retained_cols)

Generating metadata:  (219, 30)
Adding release_year:  (219, 31)
Matching data based on original dataset:  (64, 45)
Dropping duplicate values:  (64, 45)


<ipython-input-15-00d7ac0c5e44>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_artist_genres'] = dataset['artist_features.genres'].apply(lambda x: len(x))
<ipython-input-15-00d7ac0c5e44>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_track_available_markets'] = dataset['track_info.available_markets'].apply(lambda x: len(x))
<ipython-input-15-00d7ac0c5e44>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

## 2.5. Loosening requirement for matching

In [ ]:
def spotify_preporcess_loose(dataset, original_df, retained_cols):
  dataset = dataset[retained_cols]
  # Generating metadata
  dataset['num_artist_genres'] = dataset['artist_features.genres'].apply(lambda x: len(x))
  dataset['num_track_available_markets'] = dataset['track_info.available_markets'].apply(lambda x: len(x))
  dataset['num_album_available_markets'] = dataset['track_info.album.available_markets'].apply(lambda x: len(x))
  print('Generating metadata: ',dataset.shape)
  # Creating release_year col
  dataset['release_year'] = dataset['track_info.album.release_date'].str.extract(r'^(\d{4})')

  # Convert the 'release_year' column to numeric
  dataset.loc[:, 'release_year'] = pd.to_numeric(dataset['release_year'], errors='coerce')
  print('Adding release_year: ', dataset.shape)
  # Merge df_test and train_set based on all three conditions
  matched_rows = dataset.merge(original_df,
                             how='inner',
                             left_on=['artist_features.name',
                                      #'release_year',
                                      'track_info.name'
                                      ],
                             right_on=['artist',
                                       #'year',
                                       'title'
                                       ])
  print('Matching data based on original dataset: ', matched_rows.shape)
  # Dropping duplicate rows
  matched_rows= matched_rows.drop_duplicates(subset=[
    col for col in matched_rows.columns
    if col not in ['artist_features.genres', 'track_info.available_markets', 'track_info.album.available_markets']])
  print('Dropping duplicate values: ', matched_rows.shape)
  return matched_rows,dataset

In [ ]:
train_matched_r2_loose, train_r2_raw_loose = spotify_preporcess_loose(train_set_r2_modified, train_r2, retained_cols)

Generating metadata:  (848, 30)
Adding release_year:  (848, 31)
Matching data based on original dataset:  (524, 46)
Dropping duplicate values:  (517, 46)


<ipython-input-37-fe25bee8fffc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_artist_genres'] = dataset['artist_features.genres'].apply(lambda x: len(x))
<ipython-input-37-fe25bee8fffc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_track_available_markets'] = dataset['track_info.available_markets'].apply(lambda x: len(x))
<ipython-input-37-fe25bee8fffc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [ ]:
train_matched_r2_loose['title'].nunique()

248

# 3.Crawling from a created playlist

In [ ]:
train_r2['title']

0              My Happiness
1          Unchained Melody
2             Woman in Love
3                   Grenade
4                   No Time
               ...         
350    A World Without Love
351          But Not For Me
352               Surf City
353        It's Gonna Be Me
354         In The Army Now
Name: title, Length: 355, dtype: object

In [ ]:
# showing all columns
pd.set_option('display.max_rows', None)

In [ ]:
train_r3 = train_r2[~train_r2['title'].isin(train_matched_r2_modified['title'])].copy()
train_r3.reset_index(drop=True, inplace=True)
train_r3

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top_genre
0,2,Unchained Melody,The Teddy Bears,2011,114,44,53,-8,13,47,139,49,3,37,pop
1,9,Someone Elses Roses,Joan Regan,2019,100,14,29,-10,15,32,143,89,3,34,adult standards
2,10,You Belong to My Heart,Timi Yuro,2013,108,39,46,-10,16,50,126,82,3,35,adult standards
3,13,Laugh Laugh,The Beau Brummels,2006,129,68,49,-9,10,75,177,30,5,30,brill building pop
4,14,What Do You Want,Adam Faith,1989,147,37,55,-9,24,85,98,84,4,35,brill building pop
5,18,"Beauty and the Beast - from the Soundtrack ""Be...",CÃ©line Dion,1992,77,40,46,-10,13,12,249,64,3,65,canadian pop
6,19,Heart Full of Soul,The Yardbirds,2002,141,67,49,-10,5,70,148,23,4,62,album rock
7,23,Fading Like a Flower (Every Time You Leave),Roxette,1991,105,67,61,-6,22,32,231,20,3,64,europop
8,27,Drinks On The House,Green Martini Keys,2016,97,16,57,-16,11,15,181,100,5,28,pop
9,28,Rock Your Baby,George McCrae,2001,104,77,59,-11,7,95,201,31,4,55,disco


In [ ]:
train_r3.to_csv('train_r3.csv')

In [ ]:
train_r3_loose = train_r2[~train_r2['title'].isin(train_matched_r2_loose['title'])].copy()
train_r3_loose.reset_index(drop=True, inplace=True)
train_r3_loose

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top_genre
0,9,Someone Elses Roses,Joan Regan,2019,100,14,29,-10,15,32,143,89,3,34,adult standards
1,18,"Beauty and the Beast - from the Soundtrack ""Be...",CÃ©line Dion,1992,77,40,46,-10,13,12,249,64,3,65,canadian pop
2,23,Fading Like a Flower (Every Time You Leave),Roxette,1991,105,67,61,-6,22,32,231,20,3,64,europop
3,27,Drinks On The House,Green Martini Keys,2016,97,16,57,-16,11,15,181,100,5,28,pop
4,36,Ninety-Nine Ways,Tab Hunter,2000,125,33,59,-10,9,68,108,87,3,30,deep adult standards
5,37,Angel of Mine - Radio Mix,Monica,1998,90,56,59,-7,22,21,250,0,2,66,dance pop
6,42,"Because You Loved Me (Theme from ""Up Close and...",CÃ©line Dion,1996,120,47,61,-10,10,18,274,31,3,74,canadian pop
7,45,Free - Mood II Swing Radio Edit,Ultra NatÃ©,1997,125,92,76,-6,4,74,233,0,4,60,deep house
8,63,I Gotta Feeling,The Black Eyed Peas,2009,128,77,74,-6,51,61,289,9,3,77,dance pop
9,64,Sh-Boom - Single Version,The Chords,2007,137,47,67,-9,27,91,144,85,5,52,doo-wop


In [ ]:
train_r3_loose.to_csv('train_r3_loose.csv')

In [ ]:
test_r3 = test_r2[~test_r2['title'].isin(test_r2_matched_modified['title'])].copy()
test_r3.reset_index(drop=True, inplace=True)
test_r3

,Id,title,artist,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,454,Pump It,The Black Eyed Peas,2005,154,93,65,-3,75,74,213,1,18,72
1,455,"Circle of Life - From ""The Lion King""/Soundtra...",Elton John,1994,161,39,30,-15,11,14,292,26,3,59
2,457,Insomnia - Radio Edit,Faithless,2010,127,92,71,-9,37,53,216,6,4,50
3,461,Collette,Billy Fury,1982,155,44,55,-10,10,73,112,82,4,27
4,464,In The Still Of The Night,The Five Satins,2010,110,32,44,-10,11,20,184,88,4,44
5,466,Don't Break The Heart That Loves You,Connie Francis,1996,73,41,39,-6,20,20,182,82,3,37
6,468,Release Me,Agnes,2009,128,92,62,-3,11,72,256,1,3,65
7,469,Rock You Like a Hurricane,Scorpions,2017,124,83,58,-6,32,67,254,0,4,61
8,471,Everytime We Touch - Radio Edit,Cascada,2011,142,98,64,-5,37,49,199,0,6,64
9,474,You're The One That I Want - From â€œGreaseâ€,John Travolta,1978,107,61,76,-11,10,82,170,28,6,74


In [ ]:
test_r3.to_csv('test_r3.csv')

In [ ]:
train_raw_r3 = pd.DataFrame()
train_playlist = '1dbT3PmOUwKDeAcPzC0Xne'
playlist_data = crawl_playlist(train_playlist, batch_size = 10)
train_raw_r3 = pd.concat([train_raw_r3, playlist_data], ignore_index=True)

In [ ]:
train_raw_r3.to_csv('train_r4.csv')

In [ ]:
def spotify_preporcess_r3(dataset, original_df, retained_cols):
  dataset = dataset[retained_cols]
  # Generating metadata
  dataset['num_artist_genres'] = dataset['artist_features.genres'].apply(lambda x: len(x))
  dataset['num_track_available_markets'] = dataset['track_info.available_markets'].apply(lambda x: len(x))
  dataset['num_album_available_markets'] = dataset['track_info.album.available_markets'].apply(lambda x: len(x))
  print('Generating metadata: ',dataset.shape)
  # Creating release_year col
  dataset['release_year'] = dataset['track_info.album.release_date'].str.extract(r'^(\d{4})')

  # Convert the 'release_year' column to numeric
  dataset.loc[:, 'release_year'] = pd.to_numeric(dataset['release_year'], errors='coerce')
  print('Adding release_year: ', dataset.shape)
  # Merge df_test and train_set based on all three conditions
  matched_rows = dataset.merge(original_df,
                             how='inner',
                             left_on=['artist_features.name',
                                      #'release_year',
                                      #'track_info.name'
                                      ],
                             right_on=['artist',
                                       #'year',
                                       #'title'
                                       ])
  print('Matching data based on original dataset: ', matched_rows.shape)
  # Dropping duplicate rows
  matched_rows= matched_rows.drop_duplicates(subset=[
    col for col in matched_rows.columns
    if col not in ['artist_features.genres', 'track_info.available_markets', 'track_info.album.available_markets']])
  print('Dropping duplicate values: ', matched_rows.shape)
  return matched_rows,dataset

In [ ]:
train_matched_r3, train_raw_r3 = spotify_preporcess_r3(train_raw_r3, train_r3,retained_cols)

Generating metadata:  (119, 30)
Adding release_year:  (119, 31)
Matching data based on original dataset:  (112, 46)
Dropping duplicate values:  (112, 46)


<ipython-input-58-e0e21266b4f0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_artist_genres'] = dataset['artist_features.genres'].apply(lambda x: len(x))
<ipython-input-58-e0e21266b4f0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['num_track_available_markets'] = dataset['track_info.available_markets'].apply(lambda x: len(x))
<ipython-input-58-e0e21266b4f0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [ ]:
train_matched_r3.shape

(112, 46)

In [ ]:
train_r4 = train_r3[~train_r3['title'].isin(train_matched_r3['title'])].copy()
train_r4.reset_index(drop=True, inplace=True)
train_r4.shape

(61, 15)

# 4.Crawling based on artist first then lookg for tracks

In [ ]:
# Querying song based on artist first, then the name of the tracks
def get_artist_tracks(artist_name, limit = 50):
    # Search for the artist
    result = sp.search(q=artist_name, type='artist', limit=1)
    artists = result['artists']['items']
    if not artists:
        return []

    artist_id = artists[0]['id']

    # Get all tracks by the artist
    tracks = []
    offset = 0
    limit = limit  # 50 == Max limit per request
    while True:
        response = sp.artist_top_tracks(artist_id)
        tracks.extend(response['tracks'])
        offset += limit
        if len(response['tracks']) < limit:
            break

    return tracks

def search_songs_details_from_artist(test_set, batch_size = 10, limit=50):
    song_data = []

    for i in range(0, len(test_set), batch_size):
        # Get a batch of track IDs and artist IDs
        track_ids = []
        artist_ids = []

        # Prepare batch of track and artist IDs
        for j in range(i, min(i + batch_size, len(test_set))):
          artist_tracks = get_artist_tracks(test_set['artist'][j], limit)
          for track in artist_tracks:
            if track['name'] == test_set['title'][j]:
              track_ids.append(track['id'])
              artist_ids.append(track['artists'][0]['id'])
        # Get features for the batch of track and artist IDs
        track_info_batch = [sp.track(track_id) for track_id in track_ids]
        track_features_batch = sp.audio_features(track_ids) # tracks' audio features require calling a whole batch to avoid getting flagged and warning
        artist_features_batch = [sp.artist(artist_id) for artist_id in artist_ids]

        # Combine track and artist features
        for track_info, track_features, artist_features in zip(track_info_batch,track_features_batch, artist_features_batch):
            song_data.append({"track_info": track_info,"track_features": track_features, "artist_features": artist_features})

    # Convert song data into a DataFrame for easier manipulation and analysis
    return pd.json_normalize(song_data)


In [ ]:
len(playlist_tracks)

150

In [ ]:
df_test_leftover = search_songs_details_from_artist(test_leftover, batch_size = 10, limit=50)

In [ ]:
df_test_leftover

,track_info.album.album_type,track_info.album.artists,track_info.album.available_markets,track_info.album.external_urls.spotify,track_info.album.href,track_info.album.id,track_info.album.images,track_info.album.name,track_info.album.release_date,track_info.album.release_date_precision,track_info.album.total_tracks,track_info.album.type,track_info.album.uri,track_info.artists,track_info.available_markets,track_info.disc_number,track_info.duration_ms,track_info.explicit,track_info.external_ids.isrc,track_info.external_urls.spotify,track_info.href,track_info.id,track_info.is_local,track_info.name,track_info.popularity,track_info.preview_url,track_info.track_number,track_info.type,track_info.uri,track_features.danceability,track_features.energy,track_features.key,track_features.loudness,track_features.mode,track_features.speechiness,track_features.acousticness,track_features.instrumentalness,track_features.liveness,track_features.valence,track_features.tempo,track_features.type,track_features.id,track_features.uri,track_features.track_href,track_features.analysis_url,track_features.duration_ms,track_features.time_signature,artist_features.external_urls.spotify,artist_features.followers.href,artist_features.followers.total,artist_features.genres,artist_features.href,artist_features.id,artist_features.images,artist_features.name,artist_features.popularity,artist_features.type,artist_features.uri,release_year
0,album,[{'external_urls': {'spotify': 'https://open.s...,"[CA, MX, US]",https://open.spotify.com/album/2szeSQtOcJgRhDX...,https://api.spotify.com/v1/albums/2szeSQtOcJgR...,2szeSQtOcJgRhDXmTS3SIf,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Monkey Business,2005-01-01,day,15,album,spotify:album:2szeSQtOcJgRhDXmTS3SIf,[{'external_urls': {'spotify': 'https://open.s...,"[CA, MX, US]",1,213066,True,USIR10500407,https://open.spotify.com/track/6btyEL6NwUa97Ne...,https://api.spotify.com/v1/tracks/6btyEL6NwUa9...,6btyEL6NwUa97Nex9cZFvo,False,Pump It,72,None,1,track,spotify:track:6btyEL6NwUa97Nex9cZFvo,0.649,0.9310,1,-3.150,0,0.1810,0.009300,0.000000,0.751,0.7440,153.645,audio_features,6btyEL6NwUa97Nex9cZFvo,spotify:track:6btyEL6NwUa97Nex9cZFvo,https://api.spotify.com/v1/tracks/6btyEL6NwUa9...,https://api.spotify.com/v1/audio-analysis/6bty...,213067,4,https://open.spotify.com/artist/1yxSLGMDHlW21z...,None,7879320,"[dance pop, pop, pop rap]",https://api.spotify.com/v1/artists/1yxSLGMDHlW...,1yxSLGMDHlW21z4YXirZDS,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Black Eyed Peas,79,artist,spotify:artist:1yxSLGMDHlW21z4YXirZDS,2005
1,compilation,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/6ptnlw9K2gpKuoc...,https://api.spotify.com/v1/albums/6ptnlw9K2gpK...,6ptnlw9K2gpKuocxaO8F2N,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Headliners,2007-06-15,day,40,album,spotify:album:6ptnlw9K2gpKuocxaO8F2N,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",2,215640,False,GBBXH9600001,https://open.spotify.com/track/07sxccsfOMu3ZiN...,https://api.spotify.com/v1/tracks/07sxccsfOMu3...,07sxccsfOMu3ZiNG31pn4d,False,Insomnia - Radio Edit,62,https://p.scdn.co/mp3-preview/eb704d921b0897e0...,6,track,spotify:track:07sxccsfOMu3ZiNG31pn4d,0.714,0.9380,8,-7.305,1,0.0369,0.060200,0.265000,0.379,0.5300,126.822,audio_features,07sxccsfOMu3ZiNG31pn4d,spotify:track:07sxccsfOMu3ZiNG31pn4d,https://api.spotify.com/v1/tracks/07sxccsfOMu3...,https://api.spotify.com/v1/audio-analysis/07sx...,215640,4,https://open.spotify.com/artist/5T4UKHhr4HGIC0...,None,641192,"[big beat, downtempo, electronica, trip hop]",https://api.spotify.com/v1/artists/5T4UKHhr4HG...,5T4UKHhr4HGIC0VzdZQtAE,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Faithless,59,artist,spotify:artist:5T4UKHhr4HGIC0VzdZQtAE,2007
2,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/

In [ ]:
'''
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

cid = '25394a9c7ba0433abf4a25ff73f5dea9'
client_secret = '169b175347b7451e81499416006375c6'

# Initialize Spotipy with user credentials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=client_secret))

def get_track_features(track_id):
    track_info = sp.track(track_id)
    track_features = sp.audio_features(track_id)[0]
    return {**track_info, **track_features}

def get_artist_features(artist_id):
    artist_info = sp.artist(artist_id)
    # You can add more artist attributes if needed
    return artist_info

def search_song(name, artist):
    # Search for the song by name and year, limit=1 ensures we are getting the most relevant result
    result = sp.search(q=f'track:{name} artist:{artist}', type='track', limit=1)
    items = result['tracks']['items']

    if items:
        # Assuming the first track is the one we want
        return items[0]
    else:
        return None
# List to hold song data
song_data = []

for song in range(len(train_set)):
    track = search_song(train_set['title'][song],train_set['artist'][song])
    if track:
        track_id = track['id']
        artist_id = track['artists'][0]['id']  # Assuming the first artist
        track_features = get_track_features(track_id)
        artist_features = get_artist_features(artist_id)
        song_data.append({"track_features": track_features, "artist_features": artist_features})

# Convert song data into a DataFrame for easier manipulation and analysis (optional)
df = pd.json_normalize(song_data)

print(df)

'''